### Overview
This notebook converts the official Albert v2 base pretrained weights in tf-hub to be usable checkpoint, as a initialisation for pre-train Albert on custom corpus

In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

In [2]:
# reader = tf.train.NewCheckpointReader('../../project_root/ALBERT/outputs/model.ckpt-6')
# name_shape_map = reader.get_variable_to_shape_map()
trainable_variables = [('bert/embeddings/word_embeddings',[30000, 128]),('bert/embeddings/token_type_embeddings',[2, 128]),('bert/embeddings/position_embeddings',[512, 128]),('bert/embeddings/LayerNorm/beta',[128]),('bert/embeddings/LayerNorm/gamma',[128]),('bert/encoder/embedding_hidden_mapping_in/kernel',[128, 768]),('bert/encoder/embedding_hidden_mapping_in/bias',[768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/kernel',[768, 768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/bias',[768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/kernel',[768, 768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/bias',[768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/kernel',[768, 768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/bias',[768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/kernel',[768, 768]),('bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/bias',[768]),('bert/encoder/transformer/group_0/inner_group_0/LayerNorm/beta',[768]),('bert/encoder/transformer/group_0/inner_group_0/LayerNorm/gamma',[768]),('bert/encoder/transformer/group_0/inner_group_0/ffn_1/intermediate/dense/kernel',[768, 3072]),('bert/encoder/transformer/group_0/inner_group_0/ffn_1/intermediate/dense/bias',[3072]),('bert/encoder/transformer/group_0/inner_group_0/ffn_1/intermediate/output/dense/kernel',[3072, 768]),('bert/encoder/transformer/group_0/inner_group_0/ffn_1/intermediate/output/dense/bias',[768]),('bert/encoder/transformer/group_0/inner_group_0/LayerNorm_1/beta',[768]),('bert/encoder/transformer/group_0/inner_group_0/LayerNorm_1/gamma',[768]),('bert/pooler/dense/kernel',[768, 768]),('bert/pooler/dense/bias',[768]),('cls/predictions/transform/dense/kernel',[768, 128]),('cls/predictions/transform/dense/bias',[128]),('cls/predictions/transform/LayerNorm/beta',[128]),('cls/predictions/transform/LayerNorm/gamma',[128]),('cls/predictions/output_bias',[30000]),('cls/seq_relationship/output_weights',[2, 768]),('cls/seq_relationship/output_bias',[2])]


#### Convert tf-hub module to checkpoint

In [3]:
"""
Convert tf-hub module to checkpoint files.
"""
albert_module = hub.Module(
    "https://tfhub.dev/google/albert_base/2",
    trainable=True)
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, '../model_ckpt/albert_base')

 

'../model_ckpt/albert_base'

#### Adjustment on checkpoint converted from tf_hub module 
1. add adam_m and adam_v variables
    
    To reduce model size, all the adam_m and adam_v variables are removed, hence we need to add them back to pretrain with variables initialized with pretrained albert model

2. For variables in the published model are stored as 'module/cls/XXX' or 'module/bert/XXX', however the expected trainable name is 'cls/XXXX' or 'bert/XXX'

3. In the published model, variables 'cls/seq_relationship/output_bias', 'cls/seq_relationship/output_bias' are missing, why??

4. Add glonal_step variable (requires to be int64 type)

In [5]:
with tf.Session() as sess:
    reader = tf.train.NewCheckpointReader('../model_ckpt/albert_base')
    name_shape_map = reader.get_variable_to_shape_map()
    new_variable_map = {}    
    for var_name in name_shape_map:
        if var_name in dict(trainable_variables):
            tensor = reader.get_tensor(var_name)
            new_variable_map[var_name] = tf.Variable(tensor)
        elif "module/" in var_name:
            tensor = reader.get_tensor(var_name)
            new_variable_map[var_name.replace("module/","")] = tf.Variable(tensor)

    for name in dict(trainable_variables):
        new_variable_map[name+'/adam_m'] = tf.Variable(tf.zeros(tuple(dict(trainable_variables)[name]), dtype=tf.float32))
        new_variable_map[name+'/adam_v'] = tf.Variable(tf.zeros(tuple(dict(trainable_variables)[name]), dtype=tf.float32)) 

    new_variable_map['cls/seq_relationship/output_bias'] = tf.Variable(tf.zeros(2, dtype=tf.float32)) 
    new_variable_map['cls/seq_relationship/output_weights'] = tf.Variable(tf.zeros((2,768), dtype=tf.dtypes.float32)) 
    new_variable_map['global_step'] = tf.Variable(0, dtype=tf.int64) 

    saver = tf.train.Saver(new_variable_map)
    sess.run(tf.global_variables_initializer())
    saver.save(sess,  '../model_ckpt_test/albert_base')  